In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
# import seaborn as sns
# import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# %pip install evaluate rouge_score faiss-cpu bert-score

%pip install evaluate rouge_score

In [1]:
from huggingface_hub import notebook_login

notebook_login()

### Install Git-LFS to upload your model checkpoints

In [2]:
%%capture
%apt install git-lfs

In [34]:
# set device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# prevent CUDA memory fragmentation issues
os.environ["TORCH_USE_CUDA_DSA"] = "1"

# free up GPU memory before training
torch.cuda.empty_cache()

Using device: cuda


## **Import libraries**

In [3]:
import os

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch
import time

import evaluate
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
# from rouge_score import rouge_score

2025-05-18 02:51:41.990808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747536702.224045      83 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747536702.288250      83 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

## **Load dataset**

In [4]:
ds = load_dataset("316usman/research_clinical_visit_note_summarization_corpus_mts")
ds

README.md:   0%|          | 0.00/525 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/561k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/43.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/182k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1201 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 1201
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 100
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 400
    })
})

## **Load models and tokenizers**

### **T5-base**

In [16]:
# load T5 model
t5_name ='google/flan-t5-base'
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_name)

# T5 tokenizer
# parameter use_fast switches on fast tokenizer
t5_tokenizer = AutoTokenizer.from_pretrained(t5_name, use_fast=True)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [21]:
MAX_LENGTH = 512

In [23]:
sample = ds['train']['prompt'][0]
sample

'Doctor: What brings you back into the clinic today, miss? \nPatient: I came in for a refill of my blood pressure medicine. \nDoctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  \nPatient: No. \nDoctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?\nPatient: No.  \nDoctor: Great. Also, for our records, how old are you and what race do you identify yourself as?\nPatient: I am seventy six years old and identify as a white female.'

In [26]:
def preprocess_function(examples):
    start_prompt = 'Summarize the following conversation doctor-patient conversation: \n'

    prompt = [start_prompt + dialogue for dialogue in examples["prompt"]]
    
    examples['input_ids'] = t5_tokenizer(
        prompt, 
        max_length=MAX_LENGTH,
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    ).input_ids

    # set up the tokenizer for targets
    targets = [summary for summary in examples['completion']]
    
    examples['labels'] = t5_tokenizer(
        targets, 
        max_length=MAX_LENGTH,
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    ).input_ids

    return examples
    

In [27]:
# preprocess dataset
t5_tokenized_ds = ds.map(preprocess_function, batched=True)
# t5_tokenized_ds = t5_tokenized_ds.remove_columns(['prompt', 'completion'])

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [37]:
# define training arguments
training_args = TrainingArguments(
    output_dir="./flan_t5_summarization",
    per_device_train_batch_size=8,
    auto_find_batch_size=True,
    learning_rate=3e-5,
    weight_decay=0.1,
    num_train_epochs=4,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    # metric_for_best_model="rouge1", 
    report_to="none", # or "wandb" for experiment tracking
    push_to_hub=True,
)

In [42]:
# create a Trainer
trainer = Trainer(
    model=t5_model,
    tokenizer=t5_tokenizer,
    args=training_args,
    train_dataset=t5_tokenized_ds["train"],
    eval_dataset=t5_tokenized_ds["test"], 
)

/tmp/ipykernel_83/2363942018.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
# train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.218000,0.237029
2,0.244100,0.233794
3,0.258100,0.232137
4,0.255500,0.231620


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=604, training_loss=0.2510682969101217, metrics={'train_runtime': 1205.0401, 'train_samples_per_second': 3.987, 'train_steps_per_second': 0.501, 'total_flos': 3289574321160192.0, 'train_loss': 0.2510682969101217, 'epoch': 4.0})

In [44]:
# push best T5 model and tokenizer to HF
trainer.push_to_hub() 

CommitInfo(commit_url='https://huggingface.co/tn379/flan_t5_summarization/commit/ec3d85b7819a933120ce3a9042f580a934885539', commit_message='End of training', commit_description='', oid='ec3d85b7819a933120ce3a9042f580a934885539', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tn379/flan_t5_summarization', endpoint='https://huggingface.co', repo_type='model', repo_id='tn379/flan_t5_summarization'), pr_revision=None, pr_num=None)

### Tomorrow starts from here

In [45]:
# load fine-tuned model from HF
local_t5 = AutoModelForSeq2SeqLM.from_pretrained("tn379/flan_t5_summarization")
local_t5_tokenizer = AutoTokenizer.from_pretrained("tn379/flan_t5_summarization")

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [46]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def compute_rouge(predictions, references):
    results = rouge.compute(
        predictions=predictions,
        references=references,
        use_stemmer=True,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    return {k: round(v, 4) for k, v in results.items()}

In [47]:
from tqdm import tqdm

predictions = []
references = []

for example in tqdm(ds['validation']):
    input_text = example["prompt"]
    reference = example["completion"]

    input_ids = local_t5_tokenizer.encode(input_text, return_tensors="pt", truncation=True).to(local_t5.device)
    with torch.no_grad():
        output_ids = local_t5.generate(input_ids, max_new_tokens=200)
    pred = local_t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    predictions.append(pred)
    references.append(reference)

100%|██████████| 100/100 [04:47<00:00,  2.88s/it]


In [48]:
results = compute_rouge(predictions, references)
print("ROUGE scores:", results)

ROUGE scores: {'rouge1': 0.3122, 'rouge2': 0.1302, 'rougeL': 0.2474}


In [49]:
for i in range(4):
    print(f"--- Example {i+1} ---")
    print(f"Original Summary:\n{references[i]}\n")
    print(f"Predicted Summary:\n{predictions[i]}\n")

--- Example 1 ---
Original Summary:
The patient is a 26-year-old female, referred to Physical Therapy for low back pain.  The patient has a history of traumatic injury to low back.  The patient stated initial injury occurred eight years ago, when she fell at a ABC Store.  The patient stated she received physical therapy, one to two visits and received modality treatment only, specifically electrical stimulation and heat pack per patient recollection.  The patient stated that she has had continuous low-back pain at varying degrees for the past eight years since that fall.  The patient gave birth in August 2008 and since the childbirth, has experienced low back pain.  The patient also states that she fell four to five days ago, while mopping her floor.  The patient stated that she landed on her tailbone and symptoms have increased since that fall.  The patient stated that her initial physician examination with Dr. X was on 01/10/09, and has a followup appointment on 02/10/09.

Predicted 

### **Use PEFT/LORA on T5**

In [ ]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, 
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1
)

In [ ]:
# bert = evaluate.load("bertscore")

# bert_res = bert.compute(predictions=predictions, references=references, lang="en")

# avg_bertscore_f1 = sum(bert_res["f1"]) / len(bert_res["f1"])


# print(f"Avg BERTScore F1: {avg_bertscore_f1:.4f}")


# # TODO: avg for precision, recall too

In [ ]:
# %pip install git+https://github.com/google-research/bleurt.git

In [ ]:
# from bleurt import score

# # bleurt = evaluate.load("bleurt", module_type="metric")


# # predictions = ["hello there", "general kenobi"]
# # references = ["hello there", "general kenobi"]
# bleurt = evaluate.load("bleurt", module_type="metric", checkpoint="BLEURT-20")
# results = bleurt.compute(predictions=predictions, references=references)



# # checkpoint = "bleurt/BLEURT-20"

# # scorer = score.BleurtScorer(checkpoint)
# # scores = scorer.score(references=references, candidates=predictions)
# # assert isinstance(scores, list) and len(scores) == 1
# # print(scores)

# # bleurt_res = evaluate.load("bleurt", checkpoint="bleurt-20")

# # avg_bleurt = sum(bleurt_res["scores"]) / len(bleurt_res["scores"])
# # print(f"Avg BLEURT: {avg_bleurt:.4f}")

In [ ]:
# print(results, sum(results['scores'])/len(results['scores'])  )

### **BART**

In [ ]:
# load BART model
bart_name = 'facebook/bart-large-cnn'
bart_model = AutoModelForSeq2SeqLM.from_pretrained(bart_name)

# BART tokenizer
bart_tokenizer = AutoTokenizer.from_pretrained(bart_name, use_fast=True)

## **EDA**

In [ ]:
train_ds = ds['train']
val_ds = ds['validation']
test_ds = ds['test']

In [ ]:
# token length computation function
def compute_token_lengths(example):
    dialogue = example['prompt']
    summary = example['completion']
    
    # T5 tokenizer
    dialogue_len_t5 = len(t5_tokenizer(dialogue, truncation=True, padding=False)['input_ids'])
    summary_len_t5 = len(t5_tokenizer(summary, truncation=True, padding=False)['input_ids'])
    
    # BART tokenizer
    dialogue_len_bart = len(bart_tokenizer(dialogue, truncation=True, padding=False)['input_ids'])
    summary_len_bart = len(bart_tokenizer(summary, truncation=True, padding=False)['input_ids'])
    
    return {
        'dialogue_len_t5': dialogue_len_t5,
        'summary_len_t5': summary_len_t5,
        'dialogue_len_bart': dialogue_len_bart,
        'summary_len_bart': summary_len_bart
    }

In [ ]:
def process_split(split_name):
    split_dataset = ds[split_name]
    lengths_dataset = split_dataset.map(compute_token_lengths)
    
    dialogue_lengths_t5 = lengths_dataset['dialogue_len_t5']
    summary_lengths_t5 = lengths_dataset['summary_len_t5']
    dialogue_lengths_bart = lengths_dataset['dialogue_len_bart']
    summary_lengths_bart = lengths_dataset['summary_len_bart']
    
    print(f"\n--- {split_name.upper()} ---")
    print(f"T5 - Avg dialogue length: {np.mean(dialogue_lengths_t5):.2f}, Max: {np.max(dialogue_lengths_t5)}")
    print(f"T5 - Avg summary length: {np.mean(summary_lengths_t5):.2f}, Max: {np.max(summary_lengths_t5)}")
    print(f"BART - Avg dialogue length: {np.mean(dialogue_lengths_bart):.2f}, Max: {np.max(dialogue_lengths_bart)}")
    print(f"BART - Avg summary length: {np.mean(summary_lengths_bart):.2f}, Max: {np.max(summary_lengths_bart)}")
    
    # return dialogue_lengths_t5, summary_lengths_t5, dialogue_lengths_bart, summary_lengths_bart
    return dialogue_lengths_t5, summary_lengths_t5

In [ ]:
# process each split
splits = ['train', 'validation', 'test']
all_lengths = {}

for split in splits:
    all_lengths[split] = process_split(split)

In [ ]:
plt.figure(figsize=(18, 12))

for idx, split in enumerate(splits):
    dialogue_lengths_t5, summary_lengths_t5 = all_lengths[split]
    
    plt.subplot(3, 2, idx + 1)
    plt.hist(dialogue_lengths_t5, bins=10, alpha=0.6, label='Dialogue Length')
    plt.hist(summary_lengths_t5, bins=10, alpha=0.6, label='Summary Length')
    plt.title(f'Token Lengths - {split.capitalize()}', fontsize=15)
    plt.xlabel('Number of Tokens', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.legend()

    # plt.subplot(3, 2, idx*2+2)
    # plt.hist(dialogue_lengths_bart, bins=50, alpha=0.6, label='Dialogue Length (BART)')
    # plt.hist(summary_lengths_bart, bins=50, alpha=0.6, label='Summary Length (BART)')
    # plt.title(f'BART Token Lengths - {split.capitalize()}', fontsize=15)
    # plt.xlabel('Number of Tokens', fontsize=12)
    # plt.ylabel('Frequency', fontsize=12)
    # plt.legend()

plt.tight_layout()
plt.savefig('token_plot_1.png')
plt.show()

In [ ]:
# plt.figure(figsize=(18, 12))

# for idx, split in enumerate(splits):
#     dialogue_lengths_t5, summary_lengths_t5, dialogue_lengths_bart, summary_lengths_bart = all_lengths[split]
    
#     plt.subplot(3, 2, idx*2+1)
#     plt.hist(dialogue_lengths_t5, bins=50, alpha=0.6, label='Dialogue Length (T5)')
#     plt.hist(summary_lengths_t5, bins=50, alpha=0.6, label='Summary Length (T5)')
#     plt.title(f'T5 Token Lengths - {split.capitalize()}', fontsize=15)
#     plt.xlabel('Number of Tokens', fontsize=12)
#     plt.ylabel('Frequency', fontsize=12)
#     plt.legend()

#     plt.subplot(3, 2, idx*2+2)
#     plt.hist(dialogue_lengths_bart, bins=50, alpha=0.6, label='Dialogue Length (BART)')
#     plt.hist(summary_lengths_bart, bins=50, alpha=0.6, label='Summary Length (BART)')
#     plt.title(f'BART Token Lengths - {split.capitalize()}', fontsize=15)
#     plt.xlabel('Number of Tokens', fontsize=12)
#     plt.ylabel('Frequency', fontsize=12)
#     plt.legend()

# plt.tight_layout()
# plt.savefig('token_plot.png')
# plt.show()

In [ ]:
# convert HF dataset to df
train_df = pd.DataFrame(ds['train'])
val_df = pd.DataFrame(ds['validation'])
test_df = pd.DataFrame(ds['test'])

In [ ]:
df = pd.concat([train_df, val_df, test_df])

In [ ]:
print(f"Dataset shape: {df.shape}, \nNull existing: {df.isnull().sum()}")
df.head()

**=> No missing dialouges or summaries**

## **Train models on training set**

### **T5**

In [ ]:
sample = train_ds['prompt'][0]
sample

In [ ]:
# def summarize(text, max_length=150, min_length=30, num_beams=4, early_stopping=True):
#     """
#     Generates a summary for the given text using the T5 model.

#     Args:
#         text (str): The input text to summarize.
#         max_length (int): Maximum length of the generated summary.
#         min_length (int): Minimum length of the generated summary.
#         num_beams (int): Number of beams for beam search decoding.
#         early_stopping (bool): Whether to stop beam search early.

#     Returns:
#         str: The generated summary.
#     """
#     input_text = "summarize: " + text
#     input_ids = t5_tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

#     summary_ids = t5_model.generate(
#         input_ids,
#         max_length=max_length,
#         min_length=min_length,
#         num_beams=num_beams,
#         early_stopping=early_stopping
#     )

#     summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
#     return summary

# # Example usage:
# article = """
# The Daman and Diu administration on Wednesday announced that all offices under the union territory will remain closed on January 26 on the occasion of Republic Day.
# In a circular issued by the administration, all heads of offices/departments have been asked to ensure that the circular is brought to the notice of all concerned.
# Republic Day is celebrated every year on January 26 to commemorate the day when the Constitution of India came into effect in 1950.
# """
# summary = summarize(sample)
# print(summary)

In [ ]:
# tokenize function
def t5_tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '

    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["prompt"]]

    example['input_ids'] = t5_tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = t5_tokenizer(example["completion"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

In [ ]:
# apply tokenization
t5_tokenized_ds = ds.map(t5_tokenize_function, batched=True)
t5_tokenized_ds = t5_tokenized_ds.remove_columns(['prompt', 'completion'])

In [ ]:
output_dir="./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    auto_find_batch_size=True,
    logging_steps=10,
    # max_steps=1,
    eval_strategy='epoch',
    report_to="none",
)

trainer = Trainer(
    model=t5_model,
    tokenizer=t5_tokenizer,
    args=training_args,
    train_dataset=t5_tokenized_ds['train'],
    eval_dataset=t5_tokenized_ds['test']
)

trainer.train()

In [ ]:
t5_checkpoint = "/kaggle/working/results/checkpoint-302"
t5_instruct_model = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint)

In [ ]:
# evaluate T5 Qualitatively
index = 100
dialogue = test_ds[index]['prompt']
human_baseline_summary = test_ds[index]['completion']

prompt = f"""
Write a clinical note for this doctor-patient dialogue.

{dialogue}

Summary:
"""

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
t5_model = t5_model.to(device)

# Tokenize input and move to device
input_ids = t5_tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate output
t5_res = t5_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(
        max_new_tokens=200, 
        num_beams=1, 
        pad_token_id=0,
        eos_token_id=1,
        decoder_start_token_id=0
    )
)

# Decode result
t5_text_res = t5_tokenizer.decode(t5_res[0], skip_special_tokens=True)

# ==================== END OF T5 RESULT =================== #


# input_ids = t5_tokenizer(prompt, return_tensors="pt").input_ids

# t5_res = t5_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
# t5_text_res = t5_tokenizer.decode(t5_res[0], skip_special_tokens=True)

# Move model to device
t5_instruct_model = t5_instruct_model.to(device)

t5_instruct_res = t5_instruct_model.generate(
    input_ids=input_ids, 
    generation_config=GenerationConfig(
        max_new_tokens=200, 
        num_beams=1,
        pad_token_id=0,
        eos_token_id=1,
        decoder_start_token_id=0
    ))

t5_instruct_text_res = t5_tokenizer.decode(t5_instruct_res[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{t5_text_res}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{t5_instruct_text_res}')

### **Tune T5 with PEFT**

### **BART**

## **In-context learning**

# **PREVIOUS VERSION**

## **Extract embeddings from train set**

### Embed and Index train set

In [ ]:
sample = train_ds['prompt'][0]
sample

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

# load a pretrained Sentence Transformer model 
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# calculate embeddings by calling model.encode()
embeddings = embedder.encode(sample)
print(embeddings.shape)

# calculate the embedding similarities
similarities = embedder.similarity(embeddings, embeddings)
print(similarities)


# train_cnv = [example['prompt'] for example in train_ds]

# # compute embeddings
# train_embd = embedder.encode(train_cnv, convert_to_numpy=True)

# # FAISS index
# embd_dim = train_embd.shape[1]
# index = faiss.IndexFlatL2(embd_dim)
# index.add(train_embd)

In [ ]:
# retrieval function
def retrieve_similar_examples(test_conversation, k=1):
    test_embedding = embedder.encode([test_conversation], convert_to_numpy=True)
    distances, indices = index.search(test_embedding, k)
    
    examples = [train_ds[int(idx)] for idx in indices[0]]
    return examples

### Build RAP

In [ ]:
def build_rap_prompt(test_conversation, k=1):
    examples = retrieve_similar_examples(test_conversation, k=k)
    
    prompt = ""
    for ex in examples:
        prompt += f"Conversation:\n{ex['prompt']}\nSummary:\n{ex['completion']}\n\n"

    prompt += f"Now summarize:\nConversation:\n{test_conversation}\nSummary:"
    return prompt

### Run inference

In [ ]:
def summarize_with_rap(test_conversation, tokenizer, model, k=1, max_length=256):
    prompt = build_rap_prompt(test_conversation, k=k)

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_length=max_length)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return summary

In [ ]:
test_example = test_ds[0]['prompt']

# one-shot
summary = summarize_with_rap(test_example, tokenizer=t5_tokenizer, model=t5_model, k=1)
print(summary)

# few-shot
summary_fewshot = summarize_with_rap(test_example, tokenizer=t5_tokenizer, model=t5_model, k=3)
print(summary_fewshot)

In [ ]:
"""
For future use
# define a preprocessing function
# def preprocess_function(examples):
#     inputs = ["summarize: " + doc for doc in examples["prompt"]] 
#     """
#     model_inputs = t5_tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

#     # tokenize labels/desired summary with text_target
#     # with t5_tokenizer.as_target_tokenizer():
#     labels = t5_tokenizer(examples["completion"], max_length=150, truncation=True, padding="max_length", return_tensors="pt") 

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs
#     """
#     examples['input_ids'] = t5_tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt").input_ids
#     examples['labels'] = t5_tokenizer(examples["completion"], max_length=150, padding="max_length", truncation=True, return_tensors="pt").input_ids

#     return examples

# inputs = [f"Summarize the following conversation: {conversation}" for conversation in examples['prompt']]
    
    # model_inputs = t5_tokenizer(
    #     inputs,
    #     max_length=MAX_LENGTH,
    #     truncation=True,
    #     padding='max_length',
    #     return_tensors="pt"
    # ).input_ids
 
    # # Set up the tokenizer for targets
    # targets = [summary for summary in examples['completion']]
    # labels = t5_tokenizer(
    #     targets,
    #     max_length=MAX_LENGTH,
    #     truncation=True,
    #     padding='max_length',
    #     return_tensors="pt"
    # ).input_ids
 
    # model_inputs["labels"] = labels["input_ids"]
    # return model_inputs


# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# t5_model.to(device)

# # define data collator
# # data_collator = DataCollatorForSeq2Seq(t5_tokenizer, model=t5_model)

# # define evaluation metric
# rouge = evaluate.load("rouge")
"""